In [ ]:
import pretty_midi
import librosa
import numpy as np

import matplotlib.pyplot as plt

import io


# generate input data(wave format)

In [ ]:
filename = "Flower_Dance.mid"
sr = 44100

In [ ]:
midi_data = pretty_midi.PrettyMIDI(filename)

audio_data = midi_data.fluidsynth(
    sr,
#     sf2_path='MuseScore_General.sf3', # default : TimGM6mb.sf2
)

hop_len = 512
freq_w = 3
octave = 7

C = librosa.cqt(
    y, sr=sr,
    hop_length= hop_len,
    n_bins= 12*freq_w*octave,
    bins_per_octave= 12*freq_w
)

src = np.abs(C)

notes = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
y_ticks = np.arange(0,12*freq_w*octave,freq_w)
y_list=[]

for o in range(octave):
    for i in range(12):
        y_list.append(notes[i]+'%d'%(o+1))

plt.style.use('default')
fig, ax = plt.subplots(1,1,figsize=(15,10))
img = librosa.display.specshow(src, sr=sr, x_axis='time', ax=ax, hop_length= hop_len, bins_per_octave= 12*freq_w,
                               cmap='jet'
                               )
ax.set(title='Pitch axis decoration')
ax.set_yticks(y_ticks)
ax.set_yticklabels(y_list)
fig.colorbar(img, ax=ax, format="%+2d")
for i in beat_times:
    ax.axvline(i,c='k')

plt.show()

# generate target data(piano roll)

In [ ]:
piano_roll = midi_data.get_piano_roll(sr)

plt.figure(figsize=(8,4))
librosa.display.specshow(
    piano_roll, #[start_pitch:end_pitch],
    hop_length=1,
    sr=sr,
    x_axis='time',
    y_axis='cqt_note',
#     fmin=pretty_midi.note_number_to_hz(start_pitch)
)
plt.show()